# Short Cut Solver

In this notebook make a basic demonstration of the ShortCutSolver. Use simplest set-up: Cartesian grid, squared Euclidean distance for cost, dense data structures (algorithm will only solve sparse sub-problems though).

In [ ]:
# load libraries
from lib.header_notebook import *

# assume that CPLEX back-end is installed
import Solvers.ShortCutSolver as ShortCutSolver
import Solvers.OT_CPLEX as OT_CPLEX
import Solvers.ShortCutSolver_CPLEX as ShortCutSolver_CPLEX

%matplotlib inline

## Problem Setup

In [ ]:
# load two 64x64 images
imgX=sciio.loadmat("data/density-img/f-000-064.mat")["a"]
imgY=sciio.loadmat("data/density-img/f-001-064.mat")["a"]

# preprocessing (add small constant background mass, normalize masses, extract geometric positions of pixels)
(muX,posX)=OTTools.processDensity_Grid(imgX,totalMass=1.,constOffset=1E-7)
(muY,posY)=OTTools.processDensity_Grid(imgY,totalMass=1.,constOffset=1E-7)

In [ ]:
# visualize images
fig=plt.figure(figsize=(8,4))
fig.add_subplot(1,2,1)
plt.imshow(imgX)
fig.add_subplot(1,2,2)
plt.imshow(imgY)
plt.tight_layout()
plt.show()

In [ ]:
# set up hierarchical partitions

# finest layer above image has 2^partitionDepth grid points per dimension, then one below is image
partitionDepth=5
# another partition parameter, to be discussed later
partitionChildMode=HierarchicalPartition.THPMode_Grid

# create partitions from point clouds & measures, export partitions already to c++ library for later use
(partitionX,pointerX)=HierarchicalPartition.GetPartition(posX,partitionDepth,partitionChildMode,imgX.shape, mu=muX,\
    signal_pos=True, signal_radii=False,clib=SolverCFC, export=True, verbose=False)

(partitionY,pointerY)=HierarchicalPartition.GetPartition(posY,partitionDepth,partitionChildMode,imgY.shape, mu=muY,\
    signal_pos=True, signal_radii=True,clib=SolverCFC, export=True, verbose=False)

pointerYpos=HierarchicalPartition.getSignalPointer(partitionY,"pos")
pointerYradii=HierarchicalPartition.getSignalPointer(partitionY,"radii", lBottom=partitionY.nlayers-2)


# for demonstration purposes: compute dense cost functions at all levels
p=2. # exponent for Euclidean distance
cList=HierarchicalPartition.GetHierarchicalCost(partitionX, partitionY,\
    lambda posx, posy : OTTools.getEuclideanCostFunction(posx,posy,p=p))

# print a few stats on the created problem
print("cells in partition x: ", partitionX.cardLayers)
print("cells in partition y: ", partitionY.cardLayers)
print("dimensions of dense costs: ",[c.shape for c in cList])

## Solving

### Multiscale Solving

In [ ]:
# the algorithm has a modular structure. different components can be combined to final algorithm

# refinement:
#     generate initial fine coupling support when doing a layer refinement
methodSetup_Refinement=ShortCutSolver.getMethodSetup_Refinement(pointerX,pointerY,SolverCFC)

# coupling handler:
#     data structure for handling cost function and coupling.
#     in this small example use naive dense data structures (so no memory will be saved, only run-time)
#     for a sparse data structure see other example files.

methodSetup_CouplingHandler=ShortCutSolver.getMethodSetup_CouplingHandler_SemiDense()

# solver for sparse sub-problems
#     in this example only use CPLEX solver (Lemon requires some more preprocessing of densities)
#     couplingHandlerType must match the coupling handler chosen above (nothing to worry about now)
#     initializeBases=True indicates that warm-starting the solver during iterations on same scale will be used.
methodSetup_SubSolver=ShortCutSolver_CPLEX.getMethodSetup_SubSolver_CPLEX(\
        couplingHandlerType=ShortCutSolver_CPLEX.CH_SemiDense,initializeBases=True)


# shielding
#     one must choose a shielding method that matches the geometry of the cost
#     in this example we have the simplest possible case: squared Euclidean distance on regular Cartesian grid.
#     so can choose simplest (and fastest) shielding method

methodSetup_Shielding=ShortCutSolver.getMethodSetup_Shielding_Grid()

In [ ]:
# do multi-scale solving.
#     algorithm gets all the chosen methods above and combines them into full ShortCut solver.
#     solves successively from very coarse level to finest level
#     is configured in verbose mode. at each level a small report is printed
time1=datetime.datetime.now()
result=ShortCutSolver.MultiscaleSolver(partitionX, partitionY, cList,\
    methodSetup_Refinement, methodSetup_CouplingHandler, methodSetup_SubSolver, methodSetup_Shielding,\
    nLayerInitial=1,nLayerFinal=None,\
    Verbose=True,\
    maxSteps=100,collectReports=True,measureTimes=True,stepwiseAnalysis=False)
time2=datetime.datetime.now()
print(time2-time1)

#### Verify Shielding Condition and Optimality

In [ ]:
# extract final neighbourhood and support from solver
xVars=ShortCutSolver.SolverGetXVars(result[0]["pointer"])
xSupport=ShortCutSolver.SolverGetSupport(result[0]["pointer"])

In [ ]:
# verify shielding condition

## do some preprocessing to gather required information

# extract some assignment map
xMap=xSupport[0][xSupport[1][:-1]]

# generate neighbourhoods
neighboursList=[None]
for i in range(1,partitionX.nlayers):
    neighboursList.append(ShortCutSolver.GetGridNeighbours(partitionX.dims[i]))

nLayer=partitionX.nlayers-1
shielding_info=ShortCutSolver.VerifyShielding(cList[nLayer],xVars[0],xVars[1],\
        neighboursList[nLayer][0],neighboursList[nLayer][1],xMap)

print("shielding condition test:", shielding_info[0])
print("missed shields:", shielding_info[1][0].shape[0])

In [ ]:
# test explicitly for violated constraints
constraint_info=ShortCutSolver.VerifyDualConstraints(cList[nLayer],\
    result[0]["result_subsolver"]["alpha"],result[0]["result_subsolver"]["beta"],1E-10)

print("constraint violation test:", constraint_info[0])
print("violated constraints:", constraint_info[1][0].shape[0])

#### Check Objective

In [ ]:
mu=result[0]["result_couplinghandler"]["mu"]
alpha=result[0]["result_subsolver"]["alpha"]
beta=result[0]["result_subsolver"]["beta"]

In [ ]:
# primal score
np.sum(mu*cList[-1])

In [ ]:
# dual score
np.sum(alpha*muX)+np.sum(beta*muY)

### Dense Solver

In [ ]:
nLayer=partitionX.nlayers-1
muX=partitionX.layers[nLayer]["mass"]
muY=partitionY.layers[nLayer]["mass"]

time1=datetime.datetime.now()
resultDense_CPLEX=OT_CPLEX.solveDense(cList[nLayer],muX,muY)
time2=datetime.datetime.now()
deltaTDense=(time2-time1).total_seconds()
print(time2-time1)

In [ ]:
# dense primal score
np.sum(resultDense_CPLEX["mu"]*cList[nLayer])

### Cleaning Up

In [ ]:
ShortCutSolver.SolverClose(result[0]["pointer"])
SolverCFC.Close(pointerX)
SolverCFC.Close(pointerY)